# 대회 개요

[대회 설명]
- 시간이 지남에 따라 고객 주문에 대한 익명화된 데이터를 사용하여 이전에 구매한 제품이 사용자의 다음 주문에 있을지 예측하는 최고의 모델을 찾는 대회

  즉, **대회의 목표는 어떤 제품이 사용자의 다음 주문에 있을지 예측하는 것**.
- [Kaggle Competition Page](https://www.kaggle.com/competitions/instacart-market-basket-analysis)

# 데이터 개요

[데이터 설명]
- 이 데이터셋은 **시간**에 따른 **고객 주문**을 설명하는 csv 파일들로 구성되어 있습니다.
- 이 데이터는 사용자가 **어떤 물품을 다시 주문할지 예측**하는 문제를 풀어야 합니다.
- 데이터셋은 **익명화**되어 있으며 20만 명이 넘는 instacart 사용자의 300만건 이상의 주문들을 포함하고 있습니다.
- 각 사용자별로, 4회에서 100회 사이의 주문 데이터가 제공됩니다. 또한 주문이 이루어진 주와 시간, 그리고 주문 간의 상대적인 시간을 제공합니다.
- 데이터 파일은 6개로 구성되어 있으며, 각 csv 파일에 대한 설명은 다음과 같습니다.

[csv별 설명]
1. orders.csv
    - order_id: 주문 식별자
    - user_id: 고객 식별자
    - eval_set: 주문이 속한 evaluation set
    - order_number: 고객의 주문 번호 (1 = 첫 번째, n = n번째)
    - order_dow: 주문한 요일
    - order_hour_of_day: 주문이 이루어진 날의 시간
    - days_since_prior: 마지막 주문 이후 일수, 30일로 제한(주문 번호 = 1의 경우 NA)

2. products.csv
    - product_id: 제품 식별자
    - product_name: 제품명
    - aisle_id: 제품 중분류 식별자
    - department_id: 제품 대분류 식별자

3. aisles.csv
    - aisle_id: 제품 중분류 식별자
    - aisle: 제품 중분류 명

4. deptartments.csv
    - department_id: 제품 대분류 식별자
    - department: 제품 대분류 명

5. order_products__SET.csv
    - order_id: 주문 식별자
    - product_id: 제품 식별자
    - add_to_cart_order:  각 제품이 장바구니에 추가된 순서
    - reordered: 제품이 과거에 해당 고객에 의해 주문된 경우 1, 그렇지 않으면 0

    - 여기서 SET 은 orders.csv의 eval_set 중 하나
        - "prior": 고객이 이전에 주문한 가장 최근 주문 데이터
        - "train": 대회를 위해 참가자에게 제공된 훈련 데이터
        - "test":  대회를 위해 참가자에게 제공된 테스트 데이터


# __Feature Engineering 수행 후 HuggingFace에 데이터 셋 push하여 형상관리 부분 작업__

## 라이브러리 임포트

In [9]:
import numpy as np
import pandas as pd

## 데이터 로드

In [10]:
IS_RUNNING_ON_LOCAL = True

if IS_RUNNING_ON_LOCAL:
    data_path = "./dataset/"
else:
    data_path = '/kaggle/working/data/'

In [12]:
if not IS_RUNNING_ON_LOCAL:
    import os
    import glob
    import zipfile

    # zip 압축 해제하기
    for file in glob.glob(os.path.join('/kaggle/input/instacart-market-basket-analysis/', "*.zip"), recursive=True):
        with zipfile.ZipFile(file, 'r') as z:
            z.extractall(path=data_path)
else:
    import gdown
    gdown.download(id='1otEj_HsJC1CG3rGOWHyYVW2abyuWNQFm', output=data_path+'order_products__prior.csv', quiet=False)
    gdown.download(id='1m87z10vP12OzC6Iv1WWe0VfaJyO5goJs', output=data_path+'orders.csv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1otEj_HsJC1CG3rGOWHyYVW2abyuWNQFm
From (redirected): https://drive.google.com/uc?id=1otEj_HsJC1CG3rGOWHyYVW2abyuWNQFm&confirm=t&uuid=827d4666-1447-49ef-bcea-61cf0de2bf9c
To: /Users/choiyoonseol/Downloads/UpstageAILab3_ML/dataset/order_products__prior.csv
100%|██████████| 578M/578M [00:20<00:00, 27.7MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1m87z10vP12OzC6Iv1WWe0VfaJyO5goJs
From (redirected): https://drive.google.com/uc?id=1m87z10vP12OzC6Iv1WWe0VfaJyO5goJs&confirm=t&uuid=b1daaee6-e0b1-43c2-99bc-7d13ea6bb801
To: /Users/choiyoonseol/Downloads/UpstageAILab3_ML/dataset/orders.csv
100%|██████████| 109M/109M [00:03<00:00, 36.3MB/s] 


In [13]:
prior = pd.read_csv(data_path+'order_products__prior.csv', low_memory=False)
orders = pd.read_csv(data_path+'orders.csv', low_memory=False)

In [14]:
prior.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Non-Null Count     Dtype
---  ------             --------------     -----
 0   order_id           32434489 non-null  int64
 1   product_id         32434489 non-null  int64
 2   add_to_cart_order  32434489 non-null  int64
 3   reordered          32434489 non-null  int64
dtypes: int64(4)
memory usage: 989.8 MB


In [7]:
orders.info(show_counts=True) #[TODO] orders 의 days_since_prior_order 결측치 확인 필요

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   order_id                3421083 non-null  int64  
 1   user_id                 3421083 non-null  int64  
 2   eval_set                3421083 non-null  object 
 3   order_number            3421083 non-null  int64  
 4   order_dow               3421083 non-null  int64  
 5   order_hour_of_day       3421083 non-null  int64  
 6   days_since_prior_order  3214874 non-null  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


In [8]:
#[TODO-COMPLETE] orders 의 days_since_prior_order 결측치 확인
# order_number 1, 즉 첫주문일 경우 days_since_prior_order 는 NaN
orders.loc[orders['days_since_prior_order'].isna()]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
11,2168274,2,prior,1,2,11,NaN
26,1374495,3,prior,1,1,14,NaN
39,3343014,4,prior,1,6,11,NaN
45,2717275,5,prior,1,3,12,NaN
...,...,...,...,...,...,...,...
3420930,969311,206205,prior,1,4,12,NaN
3420934,3189322,206206,prior,1,3,18,NaN
3421002,2166133,206207,prior,1,6,19,NaN
3421019,2227043,206208,prior,1,1,15,NaN


## 모델링 시 필요한 파일 정리

In [15]:
orders_train_test = orders.loc[(orders['eval_set'] == 'train') | (orders['eval_set'] == 'test')]
orders_train_test.to_csv(data_path+'orders_train_test.csv', index=False)

In [18]:
if IS_RUNNING_ON_LOCAL:
    import os

    for file_name in ['orders.csv', 'order_products__prior.csv']:
        if os.path.exists(data_path+file_name):
            os.remove(data_path+file_name)
            print(f'delete {data_path+file_name}')

delete ./dataset/orders.csv
delete ./dataset/order_products__prior.csv


## Feature Engineering

In [12]:
# [F & E] 문제 해결을 위해 고객의 이전 구매 패턴과 제품 정보를 알아야 하기 때문에 전체 주문 정보와 고객이 이전에 주문한 메타 정보 결합 
prd = pd.merge(orders, prior, on='order_id', how='outer')
print(prd.info(show_counts=True))
print(prd.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32640698 entries, 0 to 32640697
Data columns (total 10 columns):
 #   Column                  Non-Null Count     Dtype  
---  ------                  --------------     -----  
 0   order_id                32640698 non-null  int64  
 1   user_id                 32640698 non-null  int64  
 2   eval_set                32640698 non-null  object 
 3   order_number            32640698 non-null  int64  
 4   order_dow               32640698 non-null  int64  
 5   order_hour_of_day       32640698 non-null  int64  
 6   days_since_prior_order  30562630 non-null  float64
 7   product_id              32434489 non-null  float64
 8   add_to_cart_order       32434489 non-null  float64
 9   reordered               32434489 non-null  float64
dtypes: float64(4), int64(5), object(1)
memory usage: 2.7+ GB
None
order_id                        0
user_id                         0
eval_set                        0
order_number                    0
order_dow 

In [13]:
# 고객의 제품별 누적 구매 수
uxp = pd.pivot_table(data=prd, index=['user_id', 'product_id'], values='order_id', aggfunc='count')
uxp.columns = ['uxp_total_bought']
uxp = uxp.reset_index()
print(uxp.head())

   user_id  product_id  uxp_total_bought
0        1       196.0                10
1        1     10258.0                 9
2        1     10326.0                 1
3        1     12427.0                10
4        1     13032.0                 3


In [14]:
# 제품별 일회성 구매율
item_one = pd.pivot_table(data=uxp[uxp['uxp_total_bought'] == 1], index='product_id', values='uxp_total_bought', aggfunc='count')
item_one.columns = ['uxp_customers_one_shot']
print(item_one.head())
item_size = pd.pivot_table(data=uxp, index='product_id', values='user_id', aggfunc='count')
item_size.columns = ['uxp_unique_customers']
print(item_size.head())

userxproduct_var = pd.merge(item_one, item_size, how='left', left_index=True, right_on='product_id')
userxproduct_var['one_shot_ratio_product'] = userxproduct_var['uxp_customers_one_shot'] / userxproduct_var['uxp_unique_customers']
userxproduct_var = userxproduct_var.reset_index()
print(userxproduct_var.head())

uxp = pd.merge(uxp, userxproduct_var.loc[:, ['product_id', 'one_shot_ratio_product']], on='product_id', how='left')
print(uxp.head())

del [item_one, item_size, userxproduct_var]

            uxp_customers_one_shot
product_id                        
1.0                            440
2.0                             70
3.0                             38
4.0                            118
5.0                              2
            uxp_unique_customers
product_id                      
1.0                          716
2.0                           78
3.0                           74
4.0                          182
5.0                            6
   product_id  uxp_customers_one_shot  uxp_unique_customers  \
0         1.0                     440                   716   
1         2.0                      70                    78   
2         3.0                      38                    74   
3         4.0                     118                   182   
4         5.0                       2                     6   

   one_shot_ratio_product  
0                0.614525  
1                0.897436  
2                0.513514  
3                0.648352  
4    

In [15]:
# 가장 최근 주문한 5개 주문에서, 사용자가 해당 제품을 주문한 횟수 및 비율
prd['order_number_back'] = prd.groupby('user_id')['order_number'].transform(max) - prd['order_number'] + 1 
last_five = pd.pivot_table(data=prd.loc[prd['order_number_back'] <= 5], index=['user_id','product_id'], values='order_id', aggfunc='count')
last_five.columns = ['times_last5']
last_five['times_last5_ratio'] = last_five['times_last5'] / 5

uxp = pd.merge(left=uxp, right=last_five , on=['user_id', 'product_id'], how='left')
del last_five

In [38]:
#고객별 총 구매 횟수
total_orders = pd.pivot_table(data=prd, index='user_id', values='order_number', aggfunc='max')
total_orders.columns = ['total_orders']
total_orders = total_orders.reset_index()

# 고객별 제품 첫 주문번호
first_order_number = pd.pivot_table(data=prd, index=['user_id', 'product_id'], values='order_number', aggfunc='min')
first_order_number.columns = ['first_order_number']
first_order_number = first_order_number.reset_index()

uxp = pd.merge(left=uxp, right=pd.merge(total_orders, first_order_number, on='user_id', how='right'), on=['user_id', 'product_id'], how='left')
del [total_orders, first_order_number]

In [43]:
#제품별 재주문률
product_var = pd.pivot_table(data=prd, index='product_id', values='reordered', aggfunc='mean')
product_var.columns = ['reorder_ratio']
#주문에서 제품이 카트에 담기는 평균 순서
product_var['mean_add_to_cart_order'] = pd.pivot_table(data=prd, index='product_id', values='add_to_cart_order', aggfunc='mean')

uxp = pd.merge(left=uxp, right=product_var, on='product_id', how='left')
del product_var

In [46]:
# 주문별 구매 제품 수
order_size = pd.pivot_table(data=prd, index=['user_id', 'order_id'], values='product_id', aggfunc='count')
order_size.columns = ['size'] 
# 사용자별 평균 구매 제품 수
results = pd.pivot_table(data=order_size, index='user_id', values='size', aggfunc='mean')
results.columns = ['order_size_avg']   
results = results.reset_index()

uxp = pd.merge(left=uxp, right=results, on='user_id', how='left')
del [order_size, results]

## 허깅페이스에 데이터 업로드

In [75]:
from huggingface_hub import login , Repository
from huggingface_hub import create_repo, upload_file
from datasets import DatasetDict , Dataset, load_dataset

In [ ]:
token = input('huggingface access token을 입력해주세요: ')

In [ ]:
login(token)

In [59]:
# Hugging Face 저장소 생성
username = "developzest"
dataset_name = "kaggle_instacart"
repo_id = f"{username}/{dataset_name}"

# 저장소 생성
create_repo(repo_id, repo_type="dataset", exist_ok=True)

RepoUrl('https://huggingface.co/datasets/developzest/kaggle_instacart', endpoint='https://huggingface.co', repo_type='dataset', repo_id='developzest/kaggle_instacart')

In [72]:
# 데이터프레임을 Dataset으로 변환
uxp_dataset = Dataset.from_pandas(uxp).remove_columns(["__index_level_0__"])

# 하나로 합치기
final_dataset = DatasetDict({"uxp": uxp_dataset})

In [73]:
# 분할된 데이터셋 확인
print(final_dataset)

DatasetDict({
    uxp: Dataset({
        features: ['user_id', 'product_id', 'uxp_total_bought', 'one_shot_ratio_product', 'times_last5', 'times_last5_ratio', 'total_orders', 'first_order_number', 'reorder_ratio', 'mean_add_to_cart_order', 'order_size_avg'],
        num_rows: 13307953
    })
})


In [74]:
# 데이터셋 버전 업데이트 및 새로운 태그 추가 (업데이트)
final_dataset.push_to_hub(repo_id, commit_message="Preprocessing", revision="v1.0")

Uploading the dataset shards: 100%|██████████| 3/3 [00:50<00:00, 16.85s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/developzest/kaggle_instacart/commit/f8adf4227a855448d5a92c47bb951b698f4cd517', commit_message='Preprocessing', commit_description='', oid='f8adf4227a855448d5a92c47bb951b698f4cd517', pr_url=None, pr_revision=None, pr_num=None)

## 허깅페이스로부터 데이터 로드

In [76]:
# 데이터셋 로드
dataset_v1 = load_dataset(repo_id, revision="v1.0")

# 데이터셋 확인
print(dataset_v1)

Generating uxp split: 100%|██████████| 13307953/13307953 [00:04<00:00, 2766967.89 examples/s]


DatasetDict({
    uxp: Dataset({
        features: ['user_id', 'product_id', 'uxp_total_bought', 'one_shot_ratio_product', 'times_last5', 'times_last5_ratio', 'total_orders', 'first_order_number', 'reorder_ratio', 'mean_add_to_cart_order', 'order_size_avg'],
        num_rows: 13307953
    })
})
